In [1]:
import torch
from torch import optim
from torchsummary import summary
from torch.utils.data import random_split, DataLoader
from torchvision.datasets import CIFAR10, Food101
from torchvision.transforms import v2
from pathlib import Path
import wandb
import os
import sys

sys.path.append("../src")

from trainer import Trainer
from module import ResNetModule
from utils import model_size, load_from_checkpoint
from callbacks import OverfitCallback, EarlyStoppingCallback
from logger import WandbLogger
from dataset import MapDataset

In [2]:
data_path = Path("../data")
logs_path = Path("../logs")
logs_path.mkdir(exist_ok=True)

In [3]:
logger = WandbLogger(
    project_name="ImageClassification",
    config={
        "model_architecture": "ResNet9",
        "num_model_layers": 9,
        "batch_size": 1024,
        "max_epochs": 20,
        "optimizer": {
            "name": "Adam",
            "weight_decay": 1e-4
        },
        "lr_scheduler": {
            "max_lr": 0.01
        },
        "train_split": 0.7,
        "val_split": 0.3
    },
    logs_path=logs_path,
    offline=True
)

In [4]:
cpu_count = os.cpu_count()
dataset = CIFAR10(data_path, train=True, download=True)

train_dataset, val_dataset = random_split(
    dataset, [logger.config["train_split"], logger.config["val_split"]]
)

val_transforms = v2.Compose([
    # Normalize
    v2.ToImage(),
    v2.ToDtype(torch.float32, scale=True),
    v2.Normalize(mean=(0.4914, 0.4822, 0.4465), std=(0.2023, 0.1994, 0.2010))
])

train_dataset = MapDataset(train_dataset, transform=v2.Compose([
    # Data auguments
    v2.RandomCrop(size=(32, 32), padding=4, padding_mode='reflect'),
    v2.RandomHorizontalFlip(),

    val_transforms
]))

val_dataset = MapDataset(val_dataset, transform=val_transforms)

train_dataloader = DataLoader(
    train_dataset, batch_size=logger.config["batch_size"], shuffle=True, num_workers=cpu_count, pin_memory=True)
val_dataloader = DataLoader(
    val_dataset, batch_size=logger.config["batch_size"],  num_workers=cpu_count, pin_memory=True)

Files already downloaded and verified


In [5]:
callbacks = [
    # EarlyStoppingCallback(min_val_accuracy=90.0, accuracy_diff=5.0, wait_epochs=5),
    OverfitCallback(limit_batches=1, batch_size=10, max_epochs=500, augument_data=False)
]

In [6]:
module = ResNetModule(toy_model=False)
optimizer = optim.AdamW(
    params=module.model.parameters(),
    weight_decay=logger.config["optimizer"]["weight_decay"]
)

lr_scheduler = torch.optim.lr_scheduler.OneCycleLR(
    optimizer=optimizer,
    max_lr=logger.config["lr_scheduler"]["max_lr"],
    epochs=logger.config["max_epochs"],
    steps_per_epoch=len(train_dataloader)
)

In [7]:
# module.model, optimizer = load_from_checkpoint(
#     path="../logs/wandb/offline-run-20241215_132918-77n093vj/checkpoints/best.pt",
#     model=module.model,
#     optimizer=optimizer
# )

# summary(module.model, input_size=(train_dataset[0][0].shape), batch_size=logger.config["batch_size"], device="cpu")

In [8]:
trainer = Trainer(
    module=module,
    logger=logger,
    optimizer=optimizer,
    callbacks=callbacks,
    logs_path=logs_path,
    fast_dev_run=False,
    measure_time=True,
    lr_scheduler=lr_scheduler,
    lr_scheduler_on_epoch=False,
    checkpoint="best_val",
    num_workers=cpu_count
)

In [9]:
module.device

'cpu'

In [10]:
try:
    trainer.fit(train_dataloader, val_dataloader)
except KeyboardInterrupt as e:
    print("Run stopped!")
finally:
    wandb.finish()

wandb: Using wandb-core as the SDK backend.  Please refer to https://wandb.me/wandb-core for more information.


torch.float32
torch.int64


model_architecture,ToyNet()


RuntimeError: Expected floating point type for target with class probabilities, got Long

[Metrics](https://api.wandb.ai/links/sampath017/iwrrziwg)